In [2]:
import tflearn
import numpy as np
from tflearn.layers.normalization import batch_normalization as batchnorm
from sklearn.preprocessing import normalize
data_train = np.load('data_train.npy')
data_train = np.random.permutation(data_train)
X_train = normalize(data_train[:,:len(data_train[0])-2])
Y_train = data_train[:, len(data_train[0])-2:]

In [3]:
width0 = len(X_train[0])
width1 = int(2*width0)
width2 = int(width0/2)

net = tflearn.input_data(shape=[None, width0])
net = tflearn.fully_connected(batchnorm(net), width1, activation='relu')
net = tflearn.fully_connected(batchnorm(net), width2, activation='relu')
net = tflearn.fully_connected(net, 2, activation='softmax')
net    = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(X_train, Y_train, n_epoch=10, batch_size=64, show_metric=True)

Training Step: 62499  | total loss: 0.13435 | time: 35.298s
| Adam | epoch: 010 | loss: 0.13435 - acc: 0.9694 -- iter: 399936/400000
Training Step: 62500  | total loss: 0.13086 | time: 35.303s
| Adam | epoch: 010 | loss: 0.13086 - acc: 0.9694 -- iter: 400000/400000
--


In [5]:
data_test = np.load('data_test.npy')
X_test = normalize(data_test[:,:len(data_test[0])-2])
Y_test = data_test[:, len(data_test[0])-2:]


print(model.evaluate(X_train, Y_train))
print(model.evaluate(X_test, Y_test))


[0.97919750000000005]
[0.97661750000000003]


In [ ]:
y_train = []
for i in range(len(Y_train)):
    y_train += [Y_train[i][1]]

from sklearn.linear_model import LogisticRegression
skmodel = LogisticRegression()
skmodel.fit(X_train, y_train)
print(skmodel.score(X_train, y_train))